# Cuda Benchmark 

Benchmark for lattice hamiltonian computations using scipy comapred cupy

# Init

## Imports

In [ ]:
# Python
import itertools

# Data management
import numpy as np
import pandas as pd
import cupy as cp
import os

# Plotting
import matplotlib.pylab as plt
import seaborn as sns
sns.set(style="ticks", font_scale=.9, palette="muted")

# Fitting
import scipy.optimize as opt

# Project modules
import luescher_nd.utilities as ut

In [ ]:
%load_ext blackcellmagic

## Preparation

In [ ]:
n1d_max_range = np.arange(10, 100, 10)
ndim_max_range = [3,]
n_step_range = range(1, 5)
cuda_range = [True, False]

In [ ]:
data = []

if os.path.exists("benchmark.csv"):
    df = pd.read_csv("benchmark.csv", index_col=0)
else:
    df = pd.DataFrame()
    
df.tail()

## Actual Benchmark

# Summary

In [ ]:
df = (
    pd.read_csv("benchmark.csv", index_col=0)
    .sort_values(by=["cuda", "n1d_max", "n_step"])
    .set_index(["cuda", "n1d_max", "n_step"])
)
for key in ["kin", "ful", "matvec"]:
    df[key] = df.apply(lambda row: (row[key + "_mean"], row[key + "_sdev"]), axis=1)
    
df = (
    df[["kin", "ful", "matvec"]]
    .stack(level=0)
    .reset_index(level=-1)
    .rename(columns={"level_3": "dtype", 0: "val"})
    .reset_index()
)
df[["mean", "sdev"]] = df.apply(lambda row: pd.Series(row["val"]), axis=1)
df = df.drop("val", axis=1)


df.head()

In [ ]:
grid = sns.FacetGrid(
    data=df,
    col="n_step",
    row="dtype",
    hue="cuda",
    sharex=True,
    sharey="row",
    height=3,
    margin_titles=True,
    legend_out=False,
)

grid.map(plt.errorbar, "n1d_max", "mean", "sdev")
grid.fig.set_dpi(200)
grid.set(yscale="log")
grid.add_legend()

grid.fig.tight_layout(w_pad=1)

plt.show()

In [ ]:
grid.savefig("benchmark-cpu-gpu-hamiltonian.pdf", bbox_inches="tight")